In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import math
import csv

In [2]:
with open('WeightedGraph.txt', newline = '') as f:
    reader = csv.reader(f)
    wg = list(reader)
wg

[['1', '2', '5'],
 ['2', '3', '4'],
 ['2', '4', '1'],
 ['3', '4', '5'],
 ['3', '5', '5'],
 ['4', '7', '3'],
 ['4', '8', '5'],
 ['5', '6', '1'],
 ['7', '9', '5']]

In [3]:
f = open('input.txt', 'r')
weight = f.readline()
f.close()
print(weight)

5


In [4]:
supernode = []
def IsSupernode(n):
    for i in range(0, len(supernode)):
        for j in range(0, len(supernode[i])):
            if n == supernode[i][j]:
                return i
    return -1

In [5]:
superedge = []
idx = 0

for i in range(0, len(wg)):
    if wg[i][2] == weight:
        if IsSupernode(wg[i][0]) >= 0: # source node가 supernode에 존재한다면
            t = IsSupernode(wg[i][0])
            supernode[t].append(wg[i][1])
        else:
            supernode.append([])
            supernode[len(supernode)-1].append(wg[i][0])
            supernode[len(supernode)-1].append(wg[i][1])
    else:
        superedge.append([])
        superedge[idx].append(wg[i][0])
        superedge[idx].append(wg[i][1])
        superedge[idx].append(wg[i][2])
        idx += 1
            
print(supernode)
print(superedge)

[['1', '2'], ['3', '4', '5', '8'], ['7', '9']]
[['2', '3', '4'], ['2', '4', '1'], ['4', '7', '3'], ['5', '6', '1']]


In [7]:
for i in range(0, len(superedge)):
    for j in range(0, 2):
        if IsSupernode(superedge[i][j]) >= 0:
            t = IsSupernode(superedge[i][j])
            superedge[i][j] = supernode[t][0]

superedge

[['1', '3', '4'], ['1', '3', '1'], ['3', '7', '3'], ['3', '6', '1']]

In [8]:
superedge = pd.DataFrame(superedge)
superedge

,0,1,2
0,1,3,4
1,1,3,1
2,3,7,3
3,3,6,1


In [9]:
superedge.columns = ['Source', 'Target', 'Weight']
superedge

,Source,Target,Weight
0,1,3,4
1,1,3,1
2,3,7,3
3,3,6,1
